# COGS 108 - Data Checkpoint

# Names

- Victoria Thai
- Hannah Yick
- Jane Dinh
- Natasha Supangkat
- Gabriel Ramiro

<a id='research_question'></a>
# Research Question

What trends can be found between a county’s designation of a superfund site and its socioeconomic/demographic trends? Does the demographic makeup of a county influence the amount of time between Superfund designation and the completion of the remediation process?

# Dataset(s)

We will combine our datasets using the common variable of geographic location (county). By analyzing both aspects - Superfund activity and demographic breakdown - holistically, we will be able to better visualize and understand trends and potentially a relationship between the two.

1) **Superfund/National Priorities List (NPL) Sites**
- Number of observations: 1327 (as of April 26, 2021)
- Features: Region, state, site name, site ID, EPA ID, address, city, zip, county, federal facility indicator (whether or not the site is a federal site), latitude, longitude, listing date.
 - We will be focusing on the site name, county, and listing date.
- Summary: This dataset provides a comprehensive overview of the main characteristics of current Superfund sites on the National Priorities List. Geographical information (county) can help us better understand the spread of the data by location, and will allow us to combine with demographic data to explore trends. Additionally, we will use the listing date to determine the source of funding (based on the date of policy changes) as well as to measure how long the site has been active.
- Source: https://semspub.epa.gov/work/HQ/201371.pdf 

2) **2019 American Community Survey 5 Year Estimate by County**
- Number of observations: 3220 
- Features: Total population, race, per capita income median household income, poverty status by race. 
- Summary: This dataset provides demographic and socioeconomic data on each city’s population. This will help us understand population trends based on the NPL sites. 
- Source: https://www.socialexplorer.com/explore-maps



# Setup

In [135]:
#Imports 
import pandas as pd
import numpy as np

#Graphing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#Statistics
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

#Webscraping
import requests 
import bs4
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

# Read in the data and store it within a data frame
#npl_df = npl_df = pd.read_csv('https://raw.githubusercontent.com/COGS108/group011_sp21/main/All%20current%20Final%20NPL%20Sites%20(FOIA%204).csv?token=AI2VKGQIIMQPBWGAZGPKD5DASR4CO') 
gdrive_npl_id = '1oDIJ_v9s5i5THULln_C-N4o3Aiz0e7mu'
npl_df = pd.read_csv(f'https://docs.google.com/uc?id={gdrive_npl_id}&export=download', encoding='ISO-8859-1')
gdrive_acs_id = '1kNWY1Kc4RhQ9ttK7dTYtj0fhHE3BXOGh'
acs_df = pd.read_csv(f'https://docs.google.com/uc?id={gdrive_acs_id}&export=download', encoding='ISO-8859-1')

pd.options.display.max_rows = 5
pd.options.display.max_columns = 10

# Data Cleaning

### National Priorities List (NPL) Superfund Data Cleaning

> Let's take a look at the dataset we have for NPL sites:

In [136]:
npl_df

,Region,State,Site Name,Site ID,EPA ID,...,County,FF Ind,Latitude,Longitude,NPL Status Date
0,1,CT,BARKHAMSTED-NEW HARTFORD LANDFILL,100255,CTD980732333,...,LITCHFIELD,N,41.893947,-72.989337,10/4/1989
1,1,CT,BEACON HEIGHTS LANDFILL,100180,CTD072122062,...,NEW HAVEN,N,41.431950,-73.035281,9/8/1983
...,...,...,...,...,...,...,...,...,...,...,...
1325,10,WA,"WESTERN PROCESSING CO., INC.",1000662,WAD009487513,...,KING,N,47.425000,-122.241700,9/8/1983
1326,10,WA,WYCKOFF CO./EAGLE HARBOR,1000612,WAD009248295,...,KITSAP,N,47.621669,-122.516700,7/22/1987


> We removed the following columns from the dataset because they are irrelevant to our research.  

In [137]:
npl_df = npl_df.drop(["Region", "Site ID", "EPA ID", "Address", "Zip", "FF Ind", "Latitude", "Longitude"], axis=1)
npl_df

,State,Site Name,City,County,NPL Status Date
0,CT,BARKHAMSTED-NEW HARTFORD LANDFILL,BARKHAMSTED,LITCHFIELD,10/4/1989
1,CT,BEACON HEIGHTS LANDFILL,BEACON FALLS,NEW HAVEN,9/8/1983
...,...,...,...,...,...
1325,WA,"WESTERN PROCESSING CO., INC.",KENT,KING,9/8/1983
1326,WA,WYCKOFF CO./EAGLE HARBOR,BAINBRIDGE ISLAND,KITSAP,7/22/1987


***
### ACS Data Cleaning

> Now let's take a look at our dataset for demographic data from the American Community Survey (ACS):

In [138]:
acs_df

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,...,"White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined:","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level"
0,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,...,SE_A13001I_001,SE_A13001I_002,SE_A13001I_003,PCT_SE_A13001I_002,PCT_SE_A13001I_003
1,01001,05000US01001,Autauga County,"Autauga County, Alabama",al,...,40993,4558,36435,11.12,88.88
...,...,...,...,...,...,...,...,...,...,...,...
3219,72151,05000US72151,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",pr,...,16,16,0,100,0
3220,72153,05000US72153,Yauco Municipio,"Yauco Municipio, Puerto Rico",pr,...,117,59,58,50.43,49.57


> At a cursory look, the first row contains variable names we won't find useful. Hence we'll drop it then reset the index:

In [139]:
acs_df = acs_df.drop([0], axis=0)
acs_df.reset_index(drop=True, inplace=True)
acs_df

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,...,"White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined:","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level"
0,01001,05000US01001,Autauga County,"Autauga County, Alabama",al,...,40993,4558,36435,11.12,88.88
1,01003,05000US01003,Baldwin County,"Baldwin County, Alabama",al,...,174691,14680,160011,8.4,91.6
...,...,...,...,...,...,...,...,...,...,...,...
3218,72151,05000US72151,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",pr,...,16,16,0,100,0
3219,72153,05000US72153,Yauco Municipio,"Yauco Municipio, Puerto Rico",pr,...,117,59,58,50.43,49.57


> It also looks as if there are several columns/rows for which there are no data for, i.e. `NaN`, so we'll drop the columns that contain `NaN` values across all rows and vice versa: 

In [140]:
acs_df.dropna(axis = 0, how = 'all', inplace = True)
acs_df.dropna(axis = 1, how = 'all', inplace = True)

In [141]:
acs_df

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,...,"White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined:","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level"
0,01001,05000US01001,Autauga County,"Autauga County, Alabama",al,...,40993,4558,36435,11.12,88.88
1,01003,05000US01003,Baldwin County,"Baldwin County, Alabama",al,...,174691,14680,160011,8.4,91.6
...,...,...,...,...,...,...,...,...,...,...,...
3218,72151,05000US72151,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",pr,...,16,16,0,100,0
3219,72153,05000US72153,Yauco Municipio,"Yauco Municipio, Puerto Rico",pr,...,117,59,58,50.43,49.57


> While no rows were dropped, for columns we went from 155 to 111. Let's take a look at the variables we have now: 

In [142]:
print(acs_df.columns)

Index(['FIPS', 'Geographic Identifier', 'Area Name', 'Qualifying Name',
       'State Postal Abbreviation', 'Summary Level', 'Geographic Component',
       'File identification', 'Logical Record Number', 'State (FIPS Code)',
       ...
       'Hispanic or Latino Population for Whom Poverty  Status Is Determined:',
       'Hispanic or Latino Population for Whom Poverty  Status Is Determined: Income Below Poverty Level',
       'Hispanic or Latino Population for Whom Poverty  Status Is Determined: Income At or Above Poverty Level',
       '% Hispanic or Latino Population for Whom Poverty  Status Is Determined: Income Below Poverty Level',
       '% Hispanic or Latino Population for Whom Poverty  Status Is Determined: Income At or Above Poverty Level',
       'White Alone, Not Hispanic or Latino Population  for Whom Poverty Status Is Determined:',
       'White Alone, Not Hispanic or Latino Population  for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level',
       'White

> There seems to be several columns containing geographic information we won't find useful, so we'll drop those columns:

In [143]:
acs_df.drop(labels=['FIPS', 'Geographic Identifier', 'Summary Level', 'Geographic Component', 'File identification', 'Logical Record Number', 'State (FIPS Code)', 'County of current residence', 'Total Population.1', 'Population Density (Per Sq. Mile)', 'Area (Land)', 'Total Population:', 'Total Population: White Alone', 'Total Population: Black or African American Alone', 'Total Population: American Indian and Alaska Native Alone', 'Total Population: Asian Alone', 'Total Population: Native Hawaiian and Other Pacific Islander Alone', 'Total Population: Some Other Race Alone', 'Total Population: Two or More Races'], axis=1, inplace = True)

In [144]:
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,...,"White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined:","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level"
0,Autauga County,"Autauga County, Alabama",al,55380,76.79,...,40993,4558,36435,11.12,88.88
1,Baldwin County,"Baldwin County, Alabama",al,212830,86.21,...,174691,14680,160011,8.4,91.6
...,...,...,...,...,...,...,...,...,...,...,...
3218,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",pr,33499,18.64,...,16,16,0,100,0
3219,Yauco Municipio,"Yauco Municipio, Puerto Rico",pr,35428,75.1,...,117,59,58,50.43,49.57


> We're now down to 92 columns. Because our NPL dataset does not have for Puerto Rico, we're dropping the rows in this `acs_df` dataframe pertaining to Puerto Rico: 

In [145]:
acs_df.drop(index=acs_df[acs_df['State Postal Abbreviation'] == 'pr'].index, inplace=True)
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,...,"White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined:","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level"
0,Autauga County,"Autauga County, Alabama",al,55380,76.79,...,40993,4558,36435,11.12,88.88
1,Baldwin County,"Baldwin County, Alabama",al,212830,86.21,...,174691,14680,160011,8.4,91.6
...,...,...,...,...,...,...,...,...,...,...,...
3140,Washakie County,"Washakie County, Wyoming",wy,8027,89.75,...,6477,728,5749,11.24,88.76
3141,Weston County,"Weston County, Wyoming",wy,7049,97.42,...,6529,908,5621,13.91,86.09


> We ended up dropping 78 rows.<br><br>
Furthermore, because the scope of our research question only considers demographics living below the poverty level, we are dropping columns pertaining to demographics living at or above the poverty level:

In [146]:
acs_df.drop(list(acs_df.filter(regex = 'At or Above Poverty Level')), axis = 1, inplace = True)
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,...,Hispanic or Latino Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Hispanic or Latino Population for Whom Poverty Status Is Determined: Income Below Poverty Level,"White Alone, Not Hispanic or Latino Population for Whom Poverty Status Is Determined:","White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level","% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level"
0,Autauga County,"Autauga County, Alabama",al,55380,76.79,...,68,4.37,40993,4558,11.12
1,Baldwin County,"Baldwin County, Alabama",al,212830,86.21,...,1756,18.41,174691,14680,8.4
...,...,...,...,...,...,...,...,...,...,...,...
3140,Washakie County,"Washakie County, Wyoming",wy,8027,89.75,...,97,8.9,6477,728,11.24
3141,Weston County,"Weston County, Wyoming",wy,7049,97.42,...,4,15.38,6529,908,13.91


> We're now down to 68 columns. Let's take a closer look at the variable/column names alongside their indices: 

In [147]:
counter = 0
for col in acs_df.columns:
    print(counter, col)
    counter += 1

0 Area Name
1 Qualifying Name
2 State Postal Abbreviation
3 Total Population
4 % Total Population: White Alone
5 % Total Population: Black or African American Alone
6 % Total Population: American Indian and Alaska Native Alone
7 % Total Population: Asian Alone
8 % Total Population: Native Hawaiian and Other Pacific Islander Alone
9 % Total Population: Some Other Race Alone
10 % Total Population: Two or More Races
11 Median Household Income (In 2019 Inflation Adjusted Dollars)
12 Median Household Income (In 2019 Inflation Adjusted Dollars):
13 Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder
14 Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder
15 Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone  Householder
16 Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone
17 Median Household Income (In 2019 Inflation Adjusted Dollar

> Our research question won't be considering per capita income on such a granular level, so we'll leave out the racial per capita income breakdown, i.e. columns 23-31. We also have a redundant column at index 11. So we will drop that as well. For columns 32-67, we have several redundant columns because raw numbers and percentages are included. We'll be only keeping the columns containing percentage data:

In [148]:
acs_df = acs_df.iloc[:, np.r_[0:11, 12:23, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67]]

In [149]:
acs_df

,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,% Total Population: White Alone,...,% Native Hawaiian and Other Pacific Islander Alone &nbsp; Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Some Other Race Alone Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Two or More Races Population for Whom Poverty Status Is Determined: Income Below Poverty Level,% Hispanic or Latino Population for Whom Poverty Status Is Determined: Income Below Poverty Level,"% White Alone, Not Hispanic or Latino Population for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level"
0,Autauga County,"Autauga County, Alabama",al,55380,76.79,...,0,26.19,16.97,4.37,11.12
1,Baldwin County,"Baldwin County, Alabama",al,212830,86.21,...,0,26.28,10.79,18.41,8.4
...,...,...,...,...,...,...,...,...,...,...,...
3140,Washakie County,"Washakie County, Wyoming",wy,8027,89.75,...,NaN,21.91,0,8.9,11.24
3141,Weston County,"Weston County, Wyoming",wy,7049,97.42,...,0,NaN,60.24,15.38,13.91


> We're now left with 34 variables/columns--more workable than the 155 we started off with. <br><br>
Finally let's take a look at the breakdown of missing values from the resulting dataset:

In [150]:
# pd.options.display.max_rows = None

with pd.option_context('display.max_rows', 999, 'display.max_colwidth', None):
    acs_missingness_df = (acs_df.isna()
    .sum()
    .to_frame('missing_count')
    .assign(missing_ratio = lambda x: x['missing_count']/len(acs_df))
    .loc[acs_df.isna().any()] )
    print(acs_missingness_df)

                                                                                                                                        missing_count  \
Median Household Income (In 2019 Inflation Adjusted Dollars): White Alone Householder                                                               2   
Median Household Income (In 2019 Inflation Adjusted Dollars): Black or African American Alone Householder                                        1218   
Median Household Income (In 2019 Inflation Adjusted Dollars): American Indian and Alaska Native Alone  Householder                               1773   
Median Household Income (In 2019 Inflation Adjusted Dollars): Asian Alone                                                                        1772   
Median Household Income (In 2019 Inflation Adjusted Dollars): Native Hawaiian and Other Pacific Islander Alone  Householder                      2894   
Median Household Income (In 2019 Inflation Adjusted Dollars): Some Other Race Alon

> There appears to be a lot of missingness for economic data when it comes to racial breakdowns, especially for more marginal groups such as Native Indian and Hawaiian groups. This may limit our analysis moving forward if we wanted to look beyond aggregate data and further analyze differences between racial groups. 
***